# 自動化登入亞州大學校園入口

## 設定webdriver和指定頁面

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [2]:
url = 'https://cip.asia.edu.tw/login.php'

browser=webdriver.Chrome('D:\webdriver\chromedriver')
browser.get(url)
print("網址設定成功")

C:\Users\km2009\anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  This is separate from the ipykernel package so we can avoid doing imports until


網址設定成功


## 截圖

In [3]:
browser.save_screenshot('screen.jpg')
print("screenshot down!")

C:\Users\km2009\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1316: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  "type. It should end with a `.png` extension", UserWarning)


screenshot down!


## 驗證碼圖片定位並截圖
透過selenium來抓取驗證碼的位置，並利用上面的截圖來定位驗證碼位置做裁切。

In [4]:
element = browser.find_element_by_xpath("(//section[@id='main']//img)[2]")

C:\Users\km2009\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [5]:
element.location

{'x': 310, 'y': 465}

In [6]:
element.size

{'height': 30, 'width': 120}

In [7]:
left = element.location['x']
top = element.location['y']
right = left + element.size['width']
bottom = top + element.size['height']

In [8]:
left, right, top, bottom

(310, 430, 465, 495)

In [9]:
from PIL import Image
img  = Image.open('screen.jpg')

In [10]:
img = img.crop((left, top, right, bottom))

In [11]:
img = img.convert("RGB") # 處理RBGA的通道數問題，Bug:OSError: cannot write mode RGBA as JPEG <----JPEG不存在A通道

img

In [12]:
img.save('numbercheck.jpg', 'jpeg')

## 驗證碼數字處理
採用像素閥值的原理來處理雜訊，當超過這個像素閥值時，就直接將他漆成黑色，若沒有超過則直接變白色。

In [13]:
img = img.convert("L")  # 將圖片轉成灰階
pix_img = img.load()
w, h = img.size
threshold = 205 #像素閥值
for y in range(h):    #for迴圈將每個像素點進行判斷
    for x in range(w):
        if pix_img[x, y] < threshold:
            pix_img[x, y] = 0
        else:
            pix_img[x, y] = 255

In [14]:
# 將經過閥值處理的像素重新生成圖片
data = img.getdata()
w, h = img.size
black_point = 0
for x in range(1, w - 1):
    for y in range(1, h - 1):
        mid_pixel = data[w * y + x]
        if mid_pixel < 50:
            top_pixel = data[w * (y - 1) + x]
            left_pixel = data[w * y + (x - 1)]
            down_pixel = data[w * (y + 1) + x]
            right_pixel = data[w * y + (x + 1)]
            if top_pixel < 10:
                black_point += 1
            if left_pixel < 10:
                black_point += 1
            if down_pixel < 10:
                black_point += 1
            if right_pixel < 10:
                black_point += 1
            if black_point < 1:
                img.putpixel((x, y), 255)
            black_point = 0


In [15]:
img

## 使用google的文字辨識引擎(pytesseract)來進行處理
安裝步驟：https://blog.csdn.net/Castlehe/article/details/118751833

In [16]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe' #設定位址
print("pytesseract 設定ok")
result = pytesseract.image_to_string(img)
regex = '\d+'
result = result.replace(" ", "") #去除空格，以免造成有'29 1316'這種中間空一格的情形
print(result)

pytesseract 設定ok
96601




## 透過selenium將帳號密碼以及驗證碼輸至正確位置上

In [17]:
try:
   # login
   time.sleep(3)
   element = browser.find_element_by_xpath("//input[@class='inputword']")
   element.send_keys('我的學號')
   time.sleep(3)
   element = browser.find_element_by_xpath("(//input[@class='inputword'])[2]")
   element.send_keys('我的密碼')
   time.sleep(3)
   element = browser.find_element_by_xpath("(//input[@class='inputword'])[3]")
   element.send_keys(result)
#    browser.find_element_by_xpath("//ul[@class='actions']//a").click()
#    time.sleep(5)
finally:
        print("done")

C:\Users\km2009\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  after removing the cwd from sys.path.
C:\Users\km2009\anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  import sys


done


C:\Users\km2009\anaconda3\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  # Remove the CWD from sys.path while we load stuff.


In [18]:
browser.quit()